In [ ]:
%matplotlib inline

import torch

import numpy as np
from scipy import interpolate
import librosa
from soundfile import write
import music21

import random
from copy import deepcopy

import math
from itertools import permutations

from performer.models.ddsp_module import DDSP
from performer.datamodules.components.ddsp_dataset import DDSPDataset

from IPython.display import Audio, Image
from matplotlib import pyplot as plt

# import warnings
# warnings.filterwarnings("ignore")
# warnings.simplefilter("ignore")

In [ ]:
us = music21.environment.UserSettings()
us['musescoreDirectPNGPath'] = '/usr/bin/musescore'

In [ ]:
flt_ckpt = '../checkpoints/flute_longrun.ckpt'

In [ ]:
def midi_to_hz(midi: float) -> float:
    return 440. * 2**((midi - 69) / 12)

def hz_to_midi(hz: float) -> float:
    return 12 * torch.log2(hz / 440) + 69

def ratio_to_interval(ratio):
    return 12 * torch.log2(ratio)

In [ ]:
def adsr(ta, td, tr, zero, peak, sustain, dur):
    ts = dur - ta - td - tr
    
    env_a = torch.linspace(zero, peak, round(ta * 250))
    env_d = torch.linspace(peak, sustain, round(td * 250))
    env_sus = torch.ones(round(ts * 250)) * sustain
    env_rel = torch.linspace(sustain, zero, round(tr * 250))

    env = torch.cat([env_a, env_d, env_sus, env_rel]).cuda()
    
    return env

In [ ]:
def sin(ts: float, f: float):
    t = torch.arange(int(ts * 250), dtype=torch.float32, device='cuda') / 250
    result = torch.sin(2 * np.pi * f * t)
    
    return result

def sin_like(ts: torch.Tensor, f: float):
    t = torch.arange(ts.shape[-1], dtype=torch.float32, device='cuda') / 250
    result = torch.sin(2 * np.pi * f * t)
    
    return result

In [ ]:
def show(music):
    display(Image(str(music.write("lily.png"))))

In [ ]:
def add_microtone(note):
    cents = note.pitch.microtone.cents
    prefix = ''
    if cents > 0:
        prefix = '+'
    if abs(cents) >= 10:
        note.addLyric(f'{prefix}{int(np.round(cents))}', applyRaw=True)

In [ ]:
with torch.inference_mode():
    model = DDSP.load_from_checkpoint(flt_ckpt, map_location='cuda')
    model = model.to('cuda')
    model.eval()
    pass

In [ ]:
rand = random.Random(123)
beat = 0.75  # 1 beat is 0.75 seconds
fps = 250

In [ ]:
def build_measure(p1, p2):
    one = music21.note.Note(quarterLength=1/3)
    one.pitch.frequency = p1
    add_microtone(one)

    two = music21.note.Note(quarterLength=1/3)
    two.pitch.frequency = p2
    two.articulations.append(music21.articulations.Staccato())
    add_microtone(two)

    sl1 = music21.spanner.Slur([one, two])

    rest1 = music21.note.Rest(1/3)
    rest = music21.note.Rest(4)

    m01 = music21.stream.Measure(number=1)

    m01.append(music21.dynamics.Dynamic('sfz'))
    m01.append(one)
    m01.append(two)
    m01.append(sl1)
    m01.append(rest1)
    m01.append(rest)
    
    return m01

In [ ]:
mm = music21.stream.Measure()
for val in constant:
    nn = music21.note.Note()
    freq = midi_to_hz(ratio_to_interval(torch.tensor(val)) + 51)
    nn.pitch.frequency = freq
    print(nn.pitch.name, nn.pitch.midi, nn.pitch.microtone)
    mm.append(nn)
mm.show()

In [ ]:
constant = [2, 3, 5, 7, 11/2, 13/2, 17/4, 19/4]

amp = adsr(0.1, 0.7, 0.01, -100, -8, -48, 1.5 * beat)
silence = torch.ones(round(3.5 * beat * fps), device='cuda') * -100.
env = torch.cat([amp, silence], dim=-1)


s = music21.stream.Score(id='mainScore')
part0 = music21.stream.Part(id='part0')
part1 = music21.stream.Part(id='part1')


ys = []
parts = [part0, part1]
lines = [
    [(0, 1), (2, 4), (7, 4), (5, 2), (7, 5), (7, 6), (7, 6), (7, 6)],
    [(7, 5), (2, 3), (7, 3), (3, 0), (1, 0), (3, 1), (4, 5), (3, 0)],
]
for part, line in zip(parts, lines):
    oll = []
    for idx1, idx2 in line:
        with torch.inference_mode():
            p1 = midi_to_hz(ratio_to_interval(torch.tensor(constant[idx1])) + 51)
            p2 = midi_to_hz(ratio_to_interval(torch.tensor(constant[idx2])) + 51)
            mezura = build_measure(p1, p2)
            # if j % 3 == 2:
            #     mezura.append(music21.layout.SystemLayout(isNew=True))
            part.append(mezura)
            
            f0 = torch.ones_like(env) * p2
            f0[:int(beat*0.333*fps)] = p1
            y = model(f0[None, None, :], env[None, None, :])
            oll.append(y)

    ys.append(torch.cat(oll, dim=-1).cpu().numpy().squeeze())


tempo = music21.tempo.MetronomeMark(referent=1.0, number=90.0)

for part in parts:
    part.measure(1).insert(tempo)
    part.insert(0, music21.meter.TimeSignature('5/4'))
    s.insert(0, part)

f0 = midi_to_hz(torch.ones_like(env, device='cuda') * 51-12)
amp = adsr(0.1, 0.7, 0.01, -100, -3, -48, 2.5 * beat)
silence = torch.ones(round(2.5 * beat * fps), device='cuda') * -100.
env = torch.cat([amp, silence], dim=-1)
oll = []
for _ in range(8):
    with torch.inference_mode():
        y = model(f0[None, None, :], env[None, None, :])  # * (torch.randn(1, device='cuda') * 0.25 + 1))
        oll.append(y)

ys.append(torch.cat(oll, dim=-1).cpu().numpy().squeeze())

s.show()
Audio(data=sum(ys), rate=48000, normalize=True)

In [ ]:
vector = microtones.JIVector(syntonic_commas_down=2)

In [ ]:
vector.calculate_ji_markup()

In [ ]:
class HEJIAccidental:  # give a _tweaks value?
    def __init__(
        self,
        accidental_markup=None,
    ):
        self.accidental_markup = accidental_markup


class HEJIPitch:
    def __init__(
        self,
        fundamental=None,
        ratio=None,
    ):
        self.fundamental = fundamental
        self.ratio = ratio
        self.bundle = microtones.make_ji_bundle(self.fundamental, self.ratio)
        self.accidental = self._calculate_accidental_markup()
        self.et_cent_deviation_markup = self._calculate_et_cent_deviation()
        self.written_pitch = self._calculate_written_pitch()

    def __str__(self):
        s = f"{self.written_pitch}!"  # do not allow to be unforced?
        return s

    def _calculate_accidental_markup(self):
        accidental_markup = self.bundle.vector.calculate_ji_markup()
        a = HEJIAccidental(accidental_markup)
        return a

    def _calculate_et_cent_deviation(self):
        deviation = microtones.return_cent_deviation_markup(
            ratio=self.ratio,
            fundamental=self.fundamental,
            chris=False,
        )
        return deviation

    def _calculate_written_pitch(self):
        temp_note = abjad.Note(self.fundamental, (1, 4))
        microtones.tune_to_ratio(temp_note.note_head, self.ratio)
        written_pitch = temp_note.written_pitch
        return written_pitch

    def _get_lilypond_format(self):
        return str(self)


In [ ]:
p = HEJIPitch("c'", "5/4")

In [ ]:
p.accidental.accidental_markup

In [ ]:
string = "d'8 f' a' d'' f'' gs'4 r8 e' gs' b' e'' gs'' a'4"
voice = abjad.Voice(string, name="RH_Voice")
staff = abjad.Staff([voice], name="RH_Staff")
score = abjad.Score([staff], name="Score")
abjad.show(score)

In [ ]:
note = abjad.Note("c'4")
staff = abjad.Staff([note])
score = abjad.Score([staff])
leaf = abjad.get.leaf(score, 0)
abjad.attach(score, abjad.LilyPondLiteral('\tune 5', site='before'))

In [ ]:
lilypond_file = abjad.LilyPondFile(
    items=[
        "#(set-default-paper-size \"a4\" \'portrait)",
        r"#(set-global-staff-size 16)",
        "\\include \'microlily/he.ly\'",
        score,
        abjad.Block(name="layout"),
    ],
)
style = '"dodecaphonic"'
lilypond_file["layout"].items.append(fr"\accidentalStyle {style}" )

In [ ]:
abjad.show(score)

In [ ]:
abjad.attach(p.accidental.accidental_markup, voice[5])
abjad.show(score)

In [ ]:
shit = abjad.NumberedPitch(12.18)

In [ ]:
shit.

In [ ]:
p0 = music21.stream.Part(id='part0')
p0.insert(0, music21.meter.TimeSignature('5/4'))
for m in measures:
    p0.append(m)
tempo = music21.tempo.MetronomeMark(referent=1.0, number=90.0)
p0.measure(1).insert(tempo)

In [ ]:
p0.show()

In [ ]:
one = music21.note.Note(quarterLength=1/3)
one.pitch.frequency = 440*11/7
add_microtone(one)

two = music21.note.Note(quarterLength=1/3)
two.pitch.frequency = 440*11/13
two.articulations.append(music21.articulations.Staccato())
add_microtone(two)

sl1 = music21.spanner.Slur([one, two])

rest1 = music21.note.Rest(1/3)

rest = music21.note.Rest(4)

s = music21.stream.Score(id='mainScore')
p0 = music21.stream.Part(id='part0')

m01 = music21.stream.Measure(number=1)

m01.append(music21.dynamics.Dynamic('sfz'))
m01.append(one)
m01.append(two)
m01.append(sl1)
m01.append(rest1)
m01.append(rest)

for i in range(10):
    mezura = deepcopy(m01)
    if i % 3 == 2:
        mezura.append(music21.layout.SystemLayout(isNew=True))
    p0.append(mezura)

tempo = music21.tempo.MetronomeMark(referent=1.0, number=90.0)
p0.measure(1).insert(tempo)
p0.insert(0, music21.meter.TimeSignature('5/4'))
s.insert(0, p0)
s.show()

In [ ]:
one.seconds